In [1]:
import splat
import wisps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
from astropy.visualization import ZScaleInterval
import matplotlib

%matplotlib inline

In [2]:
#wisps.POLYNOMIAL_RELATIONS

In [3]:
df_ids=pd.read_pickle(wisps.LIBRARIES+'/candidates.pkl')

In [4]:
df=pd.DataFrame()

In [5]:
df['grism_id']=df_ids[0].values

In [ ]:
%%capture
df['spectra']=df.grism_id.apply(lambda x: wisps.Source(name=x.replace('g141', 'G141')))

In [ ]:
df.to_hdf(wisps.OUTPUT_FILES+'/true_spectra_cands.hdf', key='spectra_only')

In [ ]:
df=pd.read_hdf(wisps.OUTPUT_FILES+'/true_spectra_cands.hdf', key='spectra_only')

In [ ]:
#remove extra problematic spectra
problems=['cosmos-02-g141_04153', 'cosmos-05-g141_07469']

In [ ]:
df=(df[ ~df.grism_id.str.lower().isin(problems)]).reset_index(drop=True)

In [ ]:
df['spt']=df['spectra'].apply(lambda x: x.spt)

In [ ]:
df=df.sort_values(by='spt').reset_index(drop=True)

In [ ]:
def plot_image(sp, ax, cmap='inferno'):
     # mapping between filters and images
    image_data_dict={'F140W' : sp.photo_image.f140,
                'F160W': sp.photo_image.f160,
                'F110W': sp.photo_image.f110}
    try:
        image_key_to_use=[ k for k in image_data_dict.keys() if image_data_dict[k]['grid'] is not None  ][0]
    except:
        image_key_to_use='F140W'

    
    mag_in_filter=np.round(sp.mags[image_key_to_use][0])
    image_data=image_data_dict[image_key_to_use]
    
    image=image_data['data']
    
    mask=image<3.*np.nanstd(image)
    vmin, vmax=ZScaleInterval().get_limits( image[mask])
    #norm=matplotlib.colors.Normalize(vmin=vmin, vmax=vmax, clip=False)
    norm=LogNorm()
    
    ax.pcolormesh(image_data['grid'][0], image_data['grid'][1], 
                   image_data['data'], cmap=cmap,
                   vmin=vmin, vmax=vmax, rasterized=True, alpha=1.0)
    
    ax.plot(image_data['center'][0], 
             image_data['center'][1], marker='+',c='#111111', ms=30)

In [ ]:
from matplotlib import gridspec
from matplotlib.colors import LogNorm

In [ ]:
df['distance']=df.spectra.apply(lambda x: x.distance['val'].value)

In [ ]:
df['RA']=df.spectra.apply(lambda x: x.coords.ra.value)
df['DEC']=df.spectra.apply(lambda x: x.coords.dec.value)

In [ ]:
df['snr1']=df.spectra.apply(lambda x: x.snr['snr1'])

In [ ]:
fdf=df[df.spt.apply(wisps.make_spt_number) >=17.]

In [ ]:
#distance distributions
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
import astropy.units as u
coords=SkyCoord(ra=fdf.RA.values*u.deg, dec=fdf.DEC.values*u.deg, distance=fdf.distance.values*u.pc)#.transform_to(coord.Galactocentric) 

In [ ]:
x, y, z=coords.cartesian.xyz

In [ ]:
r=np.sqrt(x**2+y**2)

In [ ]:
fig, ax=plt.subplots(figsize=(6, 4))
sc=plt.scatter(r, z, c=fdf.spt.apply(splat.typeToNum), cmap='viridis', s=50)
br=plt.colorbar(sc)
plt.ylabel('z (pc)', fontsize=18)
plt.xlabel('r (pc)', fontsize=18)
plt.grid()
plt.minorticks_on()

ax.set_xlim([0, 4000])
ax.set_ylim([-3000, 5000])


#ax.tick_params(which='major',direction='inout')
#ax.tick_params(which='minor', direction='in')

br.set_ticks([20, 25, 30, 35, 40])
br.set_ticklabels(['L0', 'L5', 'T0', 'T5', 'Y0'])
plt.tight_layout()
plt.savefig(wisps.OUTPUT_FIGURES+'/candidate_distances.pdf')

In [ ]:
mags=wisps.Annotator.reformat_table(pd.DataFrame.from_records(fdf.spectra.apply(lambda x: x.mags).values))

In [ ]:
import seaborn as sns
fig, ax=plt.subplots(figsize=(12, 8), ncols=3, nrows=2, sharex='col')
ax[1][0].errorbar(mags.F160W.values,  fdf.snr1.apply(np.log10).values,fmt='o', color='#2ECC40' )
ax[1][1].errorbar(mags.F140W,   fdf.snr1.apply(np.log10),fmt='o', color='#0074D9')
ax[1][2].errorbar(mags.F110W,   fdf.snr1.apply(np.log10),fmt='o', color='#FF851B')


ax[0][0].hist(mags.F160W.values,  color='#2ECC40', bins='auto' )
ax[0][1].hist(mags.F140W,   color='#0074D9', bins='auto')
ax[0][2].hist(mags.F110W, color='#FF851B' , bins='auto')


for a in np.concatenate(ax):
    a.minorticks_on()
    
    
ax[1][0].set_xlabel('F160W', fontsize=18)
ax[1][1].set_xlabel('F140W', fontsize=18)
ax[1][2].set_xlabel('F110W', fontsize=18)

ax[1][0].set_ylabel('Log J-SNR', fontsize=18)
ax[1][1].set_ylabel('Log J-SNR', fontsize=18)
ax[1][2].set_ylabel('Log J-SNR', fontsize=18)

ax[1][2].set_xlim([18, 25])
plt.tight_layout()


plt.savefig(wisps.OUTPUT_FIGURES+'/mag_distribution_candidates.pdf')

In [ ]:
dhgjk=fdf.sort_values(by='distance')[['grism_id', 'distance', 'spt']]

In [ ]:
latc_df=pd.DataFrame()

In [ ]:
latc_df['Shortname']=fdf.spectra.apply(lambda x: x.shortname.upper())
latc_df['grism id']=fdf.spectra.apply(lambda x: x.name.upper())
latc_df['snrj']=fdf.spectra.apply(lambda x: int(round(x.snr['snr1'])))
latc_df['spt']=fdf.spectra.apply(lambda x: x.spt.upper().replace('.0', ''))
latc_df['ra']=fdf.spectra.apply(lambda x: x.ra.value)
latc_df['dec']=fdf.spectra.apply(lambda x: x.dec.value)
latc_df['f110']=fdf.spectra.apply(lambda x: round(x.mags['F110W'][0], 1))
latc_df['f110_er']=fdf.spectra.apply(lambda x: round(x.mags['F110W'][1], 1))
latc_df['f140']=fdf.spectra.apply(lambda x: round(x.mags['F140W'][0],1))
latc_df['f140_er']=fdf.spectra.apply(lambda x: round(x.mags['F140W'][1], 1))
latc_df['f160']=fdf.spectra.apply(lambda x: round(x.mags['F160W'][0], 1))
latc_df['f160_er']=fdf.spectra.apply(lambda x: round(x.mags['F160W'][1], 1))
latc_df['distance']=fdf.spectra.apply(lambda x: int(round(x.distance['val'].value)))
latc_df['distance_er']=fdf.spectra.apply(lambda x: int(round(x.distance['er'].value)))



In [ ]:
latc_df

In [ ]:
%%capture 
fold='/Users/caganze/research/wisps/figures/ltwarfs/'

ids=0
for idx, row in fdf.iterrows():
    s=row.spectra
    filename=fold+'spectrum'+str(ids)+'.jpeg'
    s.plot(save=True, filename=filename)
    ids=ids+1

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
indices_df=pd.DataFrame.from_records(df.spectra.apply(lambda x: x.indices).values)

In [ ]:
for k in indices_df.columns: df[k]= indices_df[k]

In [ ]:
df.to_hdf(wisps.OUTPUT_FILES+'/true_spectra_cands.hdf', key='with_indices')

In [ ]:
import astropy.units as u

In [ ]:
#latc_df.ra.values*u.deg

In [ ]:
sk=SkyCoord(ra=latc_df.ra.values*u.deg, dec=latc_df.dec.values*u.deg, distance=latc_df.distance.values*u.pc)

In [ ]:
norm = matplotlib.colors.Normalize(vmin=20., vmax=37.0)

In [ ]:
from matplotlib import cm

In [ ]:
#cm.viridis[latc_df.spt.apply(splat.typeToNum).values]

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='mollweide')
c=ax.scatter(sk.galactic.l.wrap_at(180*u.deg).radian, sk.galactic.b.wrap_at(180*u.deg).radian,
           c=latc_df.spt.apply(splat.typeToNum).values, cmap='viridis')
plt.xlabel("l", fontsize=18)
plt.ylabel("b", fontsize=18)
plt.grid()
br=plt.colorbar(c, orientation="horizontal")
br.set_ticks([20, 25, 30, 35, 40])
br.set_ticklabels(['L0', 'L5', 'T0', 'T5', 'Y0'])
plt.tight_layout()
plt.savefig(wisps.OUTPUT_FIGURES+'/candidate_skymap.pdf')

In [ ]:
latc_df.columns

In [ ]:
import splat.empirical as spe

In [ ]:
import numba

In [ ]:
numba.types.containers.UniTuple(numba.types.float64, 2)

In [ ]:

def compute_mag(M, d):
    return M+np.log10(d)-5

def get_mag(M, d, der):
    ds=np.random.normal(d, der, 1000)
    mags=compute_mag(M, ds)
    mean=np.nanmean(mags)
    std=np.nanstd(mags)
    return mean, std

def compute_apparent_2mass_mag(spt, dist, dist_er):
    val=np.nanmean(spt)
    MJ, err2=spe.typeToMag(val, '2MASS J', ref='dupuy')
    MH, err2=spe.typeToMag(val, '2MASS H', ref='dupuy')
    
    mj, mj_er=get_mag(MJ, dist, dist_er)
    mh, mh_er=get_mag(MH, dist, dist_er)

    return [MJ, MH, mj, mj_er, mh, mh_er]

In [ ]:
jhs=np.array([compute_apparent_2mass_mag(x, y, z) for x, y, z in zip(latc_df.spt.apply(wisps.make_spt_number).values, latc_df.distance.values, latc_df.distance_er.values)])

In [ ]:
sl=splat.searchLibrary(vlm=True, spex_type=[20, 40])

In [ ]:
sl['abs_j']=[compute_mag(M, d) for M, d in (sl[['J_2MASS', 'DISTANCE']]).applymap(float).values]
sl['abs_h']=[compute_mag(M, d) for M, d in (sl[['H_2MASS', 'DISTANCE']]).applymap(float).values]
sl['j-h']=(sl['J_2MASS']-sl['H_2MASS']).values

In [ ]:
m=plt.errorbar(jhs[:,2]-jhs[:,4], jhs[:,0], xerr=((jhs[:,3])**2+(jhs[:,5])**2)**0.5,
               fmt='o', mfc='w', mec='k')

mp=plt.scatter(jhs[:,2]-jhs[:,4], jhs[:,0], c=latc_df.spt.apply(wisps.make_spt_number).values, 
            cmap='viridis', s=200)

plt.xlabel('J-H', fontsize=18)
plt.ylabel(r'$M_J$', fontsize=18)
plt.colorbar(mp)

plt.minorticks_on()

plt.gca().invert_yaxis()
plt.savefig(wisps.OUTPUT_FILES+'/abs_mag_2mass_distribution_candidates.pdf')

In [ ]:
latc_df['2MASS j']=jhs[:,2]
latc_df['2MASS j']=jhs[:,4]
latc_df['2MASS h_ER']=jhs[:,5]
latc_df['2MASS j_ER']=jhs[:,3]


In [ ]:
latc_df.to_latex(wisps.LIBRARIES+'/candidates.tex',
             header=True, index=False, na_rep=' ')

In [ ]:
import splat
import wisps

In [ ]:
s=wisps.Source(name='cosmos-23-G141_10232')

In [ ]:
import splat

In [ ]:
splat.classifyByStandard(s.splat_spectrum, comprange=[[1.15, 1.4]])

In [ ]:
s.classify_by_standard(comprange=[[1.2, 1.6]])

In [ ]:
s.plot()

In [ ]:
 latc_df.f110.max(), latc_df.f140.max(), latc_df.f160.max()